In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import urllib

CSV_FILE_PATH = 'WA_Fn-UseC_-Telco-Customer-Churn.csv'

params = urllib.parse.quote_plus(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    'DATABASE=CustomerChurnPrediction;'
    'Trusted_Connection=yes;'
)
DB_CONNECTION_STRING = f'mssql+pyodbc:///?odbc_connect={params}'


def clean_data(df):
    """Performs necessary data cleaning and type conversions."""
    print("Cleaning data...")
    # Convert 'TotalCharges' to numeric, coercing errors to NaN
    df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
    # Fill any missing TotalCharges (perhaps for new customers) with 0
    df['TotalCharges'].fillna(0, inplace=True)

    # Convert binary string columns ('Yes'/'No') to actual booleans
    # This makes them suitable for BOOLEAN/BIT database columns.
    for col in ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn',
                'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                'StreamingTV', 'StreamingMovies', 'MultipleLines']:
        if col in df.columns:
            # Map 'Yes' to True, and everything else to False
            df[col] = df[col].apply(lambda x: True if x == 'Yes' else False)

    # Convert SeniorCitizen from 0/1 to boolean
    df['IsSeniorCitizen'] = df['SeniorCitizen'].apply(lambda x: True if x == 1 else False)
    
    # Convert Gender to 'M' or 'F'
    df['gender'] = df['gender'].str.strip().str[0]

    print("Data cleaning complete.")
    return df

           
def load_data(engine, df):
    """Transforms and loads data from the DataFrame into the database tables."""
    print("Starting data transformation and loading...")
    

    # 1. Contracts
    contract_types = df[['Contract']].drop_duplicates().reset_index(drop=True)
    contract_types.rename(columns={'Contract': 'ContractType'}, inplace=True)
    # Load data without the index, DB will create IDs
    contract_types.to_sql('Contracts', engine, if_exists='append', index=False)
    print("Loaded Contracts lookup table.")

    # 2. PaymentMethods
    payment_methods = df[['PaymentMethod']].drop_duplicates().reset_index(drop=True)
    payment_methods.rename(columns={'PaymentMethod': 'PaymentMethodName'}, inplace=True)
    # Load data without the index, DB will create IDs
    payment_methods.to_sql('PaymentMethods', engine, if_exists='append', index=False)
    print("Loaded PaymentMethods lookup table.")

    # --- Prepare Main Data by mapping foreign keys ---

    # Fetch the lookup tables back from the DB to get the auto-generated IDs
    contracts_map = pd.read_sql('SELECT * FROM Contracts', engine).set_index('ContractType')
    payment_methods_map = pd.read_sql('SELECT * FROM PaymentMethods', engine).set_index('PaymentMethodName')

    # Map the string values to their corresponding integer IDs
    df['ContractID'] = df['Contract'].map(contracts_map['ContractID'])
    df['PaymentMethodID'] = df['PaymentMethod'].map(payment_methods_map['PaymentMethodID'])

    # --- Load Main Tables ---

    # 1. Customers Table
    customers_df = df[['customerID', 'gender', 'IsSeniorCitizen', 'Partner', 'Dependents']]
    customers_df = customers_df.rename(columns={
        'customerID': 'CustomerID', 'gender': 'Gender', 'Partner': 'HasPartner', 'Dependents': 'HasDependents'
    })
    customers_df.to_sql('Customers', engine, if_exists='append', index=False)
    print("Loaded Customers tasble.")

    # 2. Accounts Table
    accounts_df = df[[
        'customerID', 'tenure', 'ContractID', 'PaymentMethodID', 'PaperlessBilling',
        'MonthlyCharges', 'TotalCharges', 'Churn'
    ]]
    accounts_df = accounts_df.rename(columns={'customerID': 'CustomerID', 'tenure': 'Tenure'})
    accounts_df.to_sql('Accounts', engine, if_exists='append', index=False)
    print("Loaded Accounts table.")
    
    # 3. CustomerServices Table
    services_df = df[[
        'customerID', 'PhoneService', 'MultipleLines', 'InternetService',
        'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport'
    ]]
    services_df = services_df.rename(columns={'customerID': 'CustomerID'})
    services_df.to_sql('CustomerServices', engine, if_exists='append', index=False)
    print("Loaded CustomerServices table.")

    # 4. EntertainmentServices Table
    entertainment_df = df[['customerID', 'StreamingTV', 'StreamingMovies']]
    entertainment_df = entertainment_df.rename(columns={'customerID': 'CustomerID'})
    entertainment_df.to_sql('EntertainmentServices', engine, if_exists='append', index=False)
    print("Loaded EntertainmentServices table.")
    
    print("All data loaded successfully!")


def main():
    """Main function to run the ETL process."""
    try:
        # Extract
        print(f"Reading data from {CSV_FILE_PATH}...")
        df = pd.read_csv(CSV_FILE_PATH)
        # Standardize column names to avoid case-sensitivity issues
        df.columns = [col.strip() for col in df.columns]

        # Clean
        df = clean_data(df)
        
        # Setup database engine
        engine = create_engine(DB_CONNECTION_STRING)
        
        
        # Transform and Load
        load_data(engine, df)

    except FileNotFoundError:
        print(f"Error: The file '{CSV_FILE_PATH}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == '__main__':
    main()




Reading data from WA_Fn-UseC_-Telco-Customer-Churn.csv...
Cleaning data...
Data cleaning complete.
Starting data transformation and loading...


C:\Users\win 10-11\AppData\Local\Temp\ipykernel_9532\1688693762.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['TotalCharges'].fillna(0, inplace=True)


Loaded Contracts lookup table.
Loaded PaymentMethods lookup table.
Loaded Customers tasble.
Loaded Accounts table.
Loaded CustomerServices table.
Loaded EntertainmentServices table.
All data loaded successfully!


In [1]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.
